In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import imageio
import os
from tqdm import tqdm_notebook 
from glob import glob
os.chdir('..')

In [ ]:
dataset_id = ''

In [2]:
dataset_id = 'quickdraw'
projection_paths = pd.read_csv('Metrics/Results/{}.csv'.format(dataset_id), index_col=0).index
names = projection_paths.str.extract(r'.*/{}-(.*).csv'.format(dataset_id), expand=False) # Clean string

In [3]:
dataset_path = '../Datasets/' + dataset_id + '/'
print(projection_paths, dataset_id)

Index(['Output/quickdraw-AE_784f_500f_500f_2000f_2f_20ep.csv',
       'Output/quickdraw-C2AE_32c_32c_32c_1568f_2f_2ep.csv',
       'Output/quickdraw-VAE_784f_2048f_1024f_512f_2f_0-25drop_10ep.csv',
       'Output/quickdraw-C2VAE_32c_64c_128c_6272f_2f_10ep.csv',
       'Output/quickdraw-tsne_s1_200p.csv',
       'Output/quickdraw-dtsne_500p_0-5l.csv', 'Output/quickdraw-pca_s1.csv',
       'Output/quickdraw-pca_s4.csv'],
      dtype='object') quickdraw


In [4]:
!mkdir {'Plots/Anim/'+dataset_id}

In [5]:
def get_projection_as_array(dataset_path):
    df = pd.read_csv(dataset_path, index_col=0)
    vs = df.values.reshape(len(df), -1, 2)
    return vs, df.index, vs.shape[1]

In [6]:
lims = {}
pos_interp = {}
indexes = {}
colors = {}
for p_i, p in enumerate(projection_paths):
    pos, ind, n_timesteps = get_projection_as_array(p)
    # Shuffle arrays
    perm = np.random.permutation(len(ind))
    pos = pos[perm]
    indexes[p] = ind[perm]

    # Set x and y axis limits
    x_max = max(pos[:,:,0].flatten())
    x_min = min(pos[:,:,0].flatten())
    y_max = max(pos[:,:,1].flatten())
    y_min = min(pos[:,:,1].flatten())
    x_max = x_max + (x_max - x_min) * .1
    x_min = x_min - (x_max - x_min) * .1
    y_max = y_max + (y_max - y_min) * .1
    y_min = y_min - (y_max - y_min) * .1
    lims[p] = (x_min, x_max, y_min, y_max)

    # Choose colors
    colormap = matplotlib.cm.Set3
    classes = indexes[p].str.split('-').str[0]
    unique_colors = {k:i for i,k in enumerate(np.unique(classes))}
    colors[p] = [colormap(unique_colors[c]) for c in classes] 

    # Add more points with akima interpolation
    n_nans = 9
    pos_int = []
    for points in pos:
        extended = []
        for po in points:
            extended.append(list(po))
            for i in range(n_nans):
                extended.append([np.nan, np.nan])
        df = pd.DataFrame(extended)
        df = df.interpolate(method='akima')
        df = df.dropna()
        pos_int.append(df.values)
    pos_interp[p] = np.array(pos_int)
    n_extended = len(df)

In [7]:
anim_dir = 'Plots/Anim/{}'.format(dataset_id)
# Draw figures
size_trail = 5
n_proj = len(projection_paths)

n_extended = 5

for t in tqdm_notebook(range(n_extended)):
    if n_proj < 4:
        fig, axes = plt.subplots(ncols=n_proj, nrows=1,figsize=(n_proj*5,5))
    else:
        ncols = 4
        nrows = (n_proj-1) // 4 + 1
        fig, axes = plt.subplots(ncols=ncols, nrows=nrows,figsize=(ncols*5,nrows*5))   
        axes = axes.flatten()
        if len(axes) > n_proj:
            [ax.set_visible(False) for ax in axes[n_proj:len(axes)]]
            
    for p_i, p in enumerate(projection_paths):
        # Draw trails
        ax = axes[p_i]
        ax.set_title(names[p_i])
        for i in range(len(indexes[p])):
            b_trail = max(0, t - n_nans * size_trail)
            e_trail = t + 1
            points = pos_interp[p][i,b_trail:e_trail]
            ax.plot(points[:,1], points[:,0], 
                    c=colors[p][i], lw=2, alpha=0.5);
        # Draw scatter
        ax.scatter(pos_interp[p][:,t,1], pos_interp[p][:,t,0], 
                   edgecolors='#000000', s=15, c=colors[p], zorder=4);
        # Write text with current t in the first axis
        if p_i == 0:
            current_t = int(t/(n_nans+1))
            ax.text(0.01, 0.01, str(current_t), 
                    ha='left', va='bottom', fontsize=18, transform=ax.transAxes)

        ax.tick_params(axis='both', which='both', bottom=False, top=False,
                       labelbottom=False, right=False, left=False, labelleft=False)
        ax.set_xlim(lims[p][0], lims[p][1])
        ax.set_ylim(lims[p][2], lims[p][3])
        plt.tight_layout()
#      plt.axis('scaled')

    # Save current frame
    plt.savefig('{}/proj-{}.png'.format(anim_dir, t),
                format='png', dpi=80, bbox_inches='tight')
#     plt.show()
    plt.clf()
    plt.close()

In [8]:
n = n_extended-1
!convert -delay 5 -loop 0 {anim_dir}/proj-%d.png[0-{n}] {anim_dir}/{dataset_id}-gif.gif

In [9]:
os.chdir(anim_dir)
! pwd

/home/eduardo/PhD/Projections/dynamic-projections/Plots/Anim/quickdraw


In [10]:
#sudo apt install mencoder

In [11]:
! ls | grep .*.png | sort -V > order.txt
! mencoder mf://@order.txt -mf fps=10:type=png -ovc x264 -x264encopts bitrate=1200:threads=2 -o {dataset_id}-mkv.mkv
! rm order.txt

MEncoder 1.2.1 (Debian), built with gcc-5.4.0 (C) 2000-2016 MPlayer Team

success: format: 16  data: 0x0 - 0x0
MF file format detected.
[mf] number of files: 5
VIDEO:  [MPNG]  0x0  24bpp  10.000 fps    0.0 kbps ( 0.0 kbyte/s)
[V] filefmt:16  fourcc:0x474E504D  size:0x0  fps:10.000  ftime:=0.1000
Opening video filter: [expand osd=1]
Expand: -1 x -1, -1 ; -1, osd: 1, aspect: 0.000000, round: 1
Opening video decoder: [ffmpeg] FFmpeg's libavcodec codec family
libavcodec version 56.60.100 (external)
Selected video codec: [ffpng] vfm: ffmpeg (FFmpeg PNG)
Could not find matching colorspace - retrying with -vf scale...
Opening video filter: [scale]
Movie-Aspect is undefined - no prescaling applied.
[swscaler @ 0x7f32c72a3fa0]bicubic scaler, from rgba to yuv420p using MMXEXT
x264 [info]: using SAR=1/1
x264 [info]: using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 AVX2 LZCNT BMI2
x264 [info]: profile High, level 3.2
Movie-Aspect is 2.01:1 - prescaling to correct movie aspect.
[swscaler